In [ ]:
import kagglehub
kagglehub.login()


Kaggle credentials set.
Kaggle credentials successfully validated.


In [ ]:
nahieu2005_medical_en_vi_corpus_path = kagglehub.dataset_download('nahieu2005/medical-en-vi-corpus')
nahieu2005_nlpmidterm_checkpoints_run1_path = kagglehub.dataset_download('nahieu2005/nlpmidterm-checkpoints-run1')
nahieu2005_vit5_finetuned_checkpoint_1_path = kagglehub.dataset_download('nahieu2005/vit5-finetuned-checkpoint-1')

print('Data source import complete.')


100%|██████████| 49.1M/49.1M [00:00<00:00, 102MB/s]

Extracting files...


100%|██████████| 3.39G/3.39G [00:38<00:00, 94.4MB/s]

Extracting files...


100%|██████████| 2.32G/2.32G [00:29<00:00, 83.5MB/s]

Extracting files...


Data source import complete.


In [ ]:
print("--- Cài đặt các thư viện cần thiết ---")
!pip install -q "transformers[torch]==4.41.2" "sacrebleu==2.4.2" "sentencepiece==0.2.0"

--- Cài đặt các thư viện cần thiết ---
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.0/58.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 71.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 84.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 96.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 96.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 61.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.8 MB/s eta 0:00:00
   ━━━━━━

In [4]:
nahieu2005_nlpprivate_path = kagglehub.dataset_download('nahieu2005/nlpprivate')
print(f"Đã tải xong private test tại: {nahieu2005_nlpprivate_path}")

100%|██████████| 189k/189k [00:00<00:00, 69.8MB/s]

Extracting files...
Đã tải xong private test tại: /root/.cache/kagglehub/datasets/nahieu2005/nlpprivate/versions/1


In [ ]:
import torch
import pandas as pd
import sacrebleu
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, T5ForConditionalGeneration, T5TokenizerFast
import zipfile
import os


print("--- Thiết lập cấu hình ---")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


FINETUNED_MODEL_PATH = os.path.join(
    nahieu2005_vit5_finetuned_checkpoint_1_path,
    "vit5-finetuned-medical-en-vi",
    "checkpoint-28125"
)
VI2EN_MODEL_CHECKPOINT = "vinai/vinai-translate-vi2en"
PRIVATE_DATA_DIR = os.path.join(nahieu2005_nlpprivate_path, "private_test")

MAX_INPUT_LENGTH = 256
MAX_TARGET_LENGTH = 256
INFERENCE_BATCH_SIZE = 8
OUTPUT_DIR = "/content/" 

print("\n--- Tải dữ liệu private test ---")
try:
    df_en = pd.read_csv(os.path.join(PRIVATE_DATA_DIR, "en.csv"), on_bad_lines='skip')
    english_private = df_en["English"].tolist()


    df_vi = pd.read_csv(os.path.join(PRIVATE_DATA_DIR, "vi.csv"), on_bad_lines='skip')
    vietnamese_private = df_vi["Vietnamese"].tolist()

    english_private = [str(s) if pd.notna(s) else "" for s in english_private]
    vietnamese_private = [str(s) if pd.notna(s) else "" for s in vietnamese_private]

    print(f"Đã tải và làm sạch {len(english_private)} câu từ tập private test.")
except Exception as e:
    print(f"Lỗi khi đọc file CSV của private test: {e}")

    print("\n5 dòng đầu của en.csv:")
    !head -n 5 "{os.path.join(PRIVATE_DATA_DIR, 'en.csv')}"
    print("\n5 dòng đầu của vi.csv:")
    !head -n 5 "{os.path.join(PRIVATE_DATA_DIR, 'vi.csv')}"
    raise

print("\n--- Tải các model và tokenizer ---")
print(f"Tải model finetuned (EN->VI) từ: {FINETUNED_MODEL_PATH}")
tokenizer_en_vi = T5TokenizerFast.from_pretrained(FINETUNED_MODEL_PATH)
model_en_vi = T5ForConditionalGeneration.from_pretrained(FINETUNED_MODEL_PATH).to(device)
model_en_vi.eval()

print(f"Tải model chuyên dịch (VI->EN) từ: {VI2EN_MODEL_CHECKPOINT}")
tokenizer_vi_en = AutoTokenizer.from_pretrained(VI2EN_MODEL_CHECKPOINT)
model_vi_en = AutoModelForSeq2SeqLM.from_pretrained(VI2EN_MODEL_CHECKPOINT).to(device)
model_vi_en.eval()

def translate_batch(texts, model, tokenizer, batch_size):
    translations = []
    with torch.no_grad():
        for i in tqdm(range(0, len(texts), batch_size), desc="Translating"):
            batch = texts[i:i + batch_size]
            inputs = tokenizer(batch, return_tensors="pt", padding=True, truncation=True, max_length=MAX_INPUT_LENGTH).to(device)
            outputs = model.generate(
                **inputs,
                max_new_tokens=MAX_TARGET_LENGTH,
                num_beams=5,
                early_stopping=True
            )
            decoded_batch = tokenizer.batch_decode(outputs, skip_special_tokens=True)
            translations.extend(decoded_batch)
    return translations

print("\n--- Bắt đầu dịch Anh -> Việt (Private Test) ---")
prefix_en_vi = "translate English to Vietnamese: "
source_en_vi = [prefix_en_vi + s for s in english_private]
vietnamese_translations = translate_batch(source_en_vi, model_en_vi, tokenizer_en_vi, INFERENCE_BATCH_SIZE)

print("\n--- Bắt đầu dịch Việt -> Anh (Private Test) ---")
english_translations = translate_batch(vietnamese_private, model_vi_en, tokenizer_vi_en, INFERENCE_BATCH_SIZE)

print("\n--- Tạo file results.csv cho Private Test ---")
results_df = pd.DataFrame({
    "English": english_translations,
    "Vietnamese": vietnamese_translations
})
csv_path = os.path.join(OUTPUT_DIR, "results.csv")
results_df.to_csv(csv_path, index=False, encoding='utf-8')
print(f"Đã lưu file kết quả tại: {csv_path}")

print("\n--- Nén file submission.zip cho Private Test ---")
zip_path = os.path.join(OUTPUT_DIR, "submission.zip")
with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
    zipf.write(csv_path, arcname="results.csv")

print(f"\n--- HOÀN TẤT! ---")
print(f"File nộp bài cho private test đã được tạo tại: {zip_path}")
print("Bạn có thể tải file này về từ cây thư mục bên trái của Colab.")

--- Thiết lập cấu hình ---

--- Tải dữ liệu private test ---
Đã tải và làm sạch 1000 câu từ tập private test.

--- Tải các model và tokenizer ---
Tải model finetuned (EN->VI) từ: /root/.cache/kagglehub/datasets/nahieu2005/vit5-finetuned-checkpoint-1/versions/1/vit5-finetuned-medical-en-vi/checkpoint-28125
Tải model chuyên dịch (VI->EN) từ: vinai/vinai-translate-vi2en


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/1.88M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.79G [00:00<?, ?B/s]


--- Bắt đầu dịch Anh -> Việt (Private Test) ---


Translating: 100%|██████████| 125/125 [04:24<00:00,  2.12s/it]



--- Bắt đầu dịch Việt -> Anh (Private Test) ---


Translating: 100%|██████████| 125/125 [02:00<00:00,  1.04it/s]


--- Tạo file results.csv cho Private Test ---
Đã lưu file kết quả tại: /content/results.csv

--- Nén file submission.zip cho Private Test ---

--- HOÀN TẤT! ---
File nộp bài cho private test đã được tạo tại: /content/submission.zip
Bạn có thể tải file này về từ cây thư mục bên trái của Colab.
